In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from enum import Enum

# Define the symbols to be used in the encryption
class Futhark(Enum):
    FEOH = 'ᚠ'
    URUZ = 'ᚢ'
    THURISAZ = 'ᚦ'
    ANSUZ = 'ᚨ'
    RAIDO = 'ᚱ'
    KAUNAN = 'ᚲ'
    GEBO = 'ᚷ'
    WUNJO = 'ᚹ'
    HAGLAZ = 'ᚺ'
    NAUDIZ = 'ᚾ'
    ISA = 'ᛁ'
    JERA = 'ᛃ'
    EIHWAZ = 'ᛇ'
    PERTHO = 'ᛈ'
    ALGIZ = 'ᛉ'
    SOWILO = 'ᛋ'
    TIWAZ = 'ᛏ'
    BERKANO = 'ᛒ'
    EHWAZ = 'ᛖ'
    MANNAZ = 'ᛗ'
    LAGUZ = 'ᛚ'
    INGWAZ = 'ᛝ'
    DAGAZ = 'ᛞ'
    OTHALA = 'ᛟ'

symbols = [symbol.value for symbol in Futhark]

# Load images from the specified directory and convert them to grayscale
def load_images(directory):
    images = []
    labels = []
    for subdir in os.listdir(directory):
        if subdir == 'RIMES':
            for filename in os.listdir(os.path.join(directory, subdir)):
                if filename.endswith('.png'):
                    img = cv2.imread(os.path.join(directory, subdir, filename))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img = cv2.resize(img, (28, 28))
                    images.append(img)
                    label = int(filename.split('-')[0])
                    labels.append(label)
        elif subdir == 'FUTHARK1':
            for filename in os.listdir(os.path.join(directory, subdir)):
                if filename.endswith('.png'):
                    img = cv2.imread(os.path.join(directory, subdir, filename))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img = cv2.resize(img, (28, 28))
                    images.append(img)
                    label = int(filename.split('_')[0])
                    labels.append(label)
        else:
            continue
    return np.array(images), np.array(labels)


# Build the convolutional neural network model
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model


# Main function
# Main function
def main():
    # Define the directory containing the image dataset
    data_dir = './images'
    
    # Load images
    images, labels = load_images(data_dir)
    num_classes = len(symbols)
    labels = to_categorical(labels, num_classes)

    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True
    )
    datagen.fit(images)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.33)
    
    # Define input shape
    input_shape = (28, 28, 1)

    # Build and compile model
    model = build_model(input_shape, num_classes)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train model
    model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

    # Evaluate model on test set
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

if __name__ == '__main__':
    main()

